In [1]:
# Author: Mike Hagenow
# Generate some templated behaviors for sanding (including corrections) to be learned through DMP-LWR

import numpy as np
import rospy

from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface

# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
def interpMultD(starting_vals,ending_vals,num_pts):
    vals = []
    for ii in range(0,num_pts):
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s for s,e in zip(starting_vals,ending_vals)))
    return np.asarray(vals).T

# returns num_vars x num_samples




Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
# Behavior one: Two passes around stringer (for testing without running sanding tool)

surfaceBSplineFile = '/home/mike/Documents/panda_uli_workspace/src/panda_uli_demo/ULIConfig/registration_models/stringer_surface_demo.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 0 # degrees
R_tool_surf = ScipyR.from_euler('y',ang_relative_to_surface,degrees=True)
q_ts = R_tool_surf.as_quat()


# Part 1: approach the starting point
segment = HybridSegment()
segment.hybrid = False
segment.num_samples = 100 # 2 seconds
segment.state_names = ['x','y','z','qx','qy','qz','qw']

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.1, 0.01)

R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()

approach_height = 0.1
starting = [approach_pt[0]+n_hat[0]*approach_height, approach_pt[1]+n_hat[1]*approach_height, approach_pt[2] +n_hat[2]*approach_height, q_app[0], q_app[1], q_app[2], q_app[3]]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.0, q_app[0], q_app[1], q_app[2], q_app[3]]
orig_vals = interpMultD(starting,ending,segment.num_samples)


# number of state variables by number of samples

segment.original_vals = []
segment.original_vals.append(orig_vals)

segments.append(segment)


# Part 2: one pass over surface (for now)
segment = HybridSegment()
segment.hybrid = True
segment.surface = surfaceBSpline
segment.num_samples = 600 # 4 seconds
segment.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw']
segment.original_vals = []


samps = int(segment.num_samples / 3)
waypoint_0 = interpMultD([0.1, 0.01, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.9, 0.01, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)
waypoint_1 = interpMultD([0.9, 0.01, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.9, 0.99, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)
waypoint_2 = interpMultD([0.9, 0.99, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.01, 0.99, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)

waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2])

segment.original_vals.append(waypoints)

segments.append(segment)


# # Part 3: move away from the surface
segment = HybridSegment()
segment.hybrid = False
segment.num_samples = 100 # 2 seconds
segment.state_names = ['x','y','z','qx','qy','qz','qw']
segment.original_vals = []

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.01, 0.99)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()


retract_height = 0.1
surf_offset = 0.005
starting = [retract_pt[0]+n_hat[0]*surf_offset, retract_pt[1]+n_hat[1]*surf_offset,retract_pt[2]+n_hat[2]*surf_offset, q_ret[0], q_ret[1], q_ret[2], q_ret[3]]
ending = [retract_pt[0]+n_hat[0]*retract_height, retract_pt[1]+n_hat[1]*retract_height,retract_pt[2]+n_hat[2]*retract_height, q_ret[0], q_ret[1], q_ret[2], q_ret[3]]
orig_vals = interpMultD(starting,ending,segment.num_samples)
segment.original_vals.append(orig_vals)

segments.append(segment)



# from cProfile import Profile
# from pstats import Stats

# p = Profile()
# p.enable()
# <stuff>
# p.disable()
# Stats(p).sort_by('cumtime').print_stats()

In [3]:
############################
# Learn Stringer model     #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('corrective_shared_autonomy')+'/../../ULIConfig/registration_models/'


model = DMPLWRhardcoded(verbose=True)
learnedSegments = model.learnModel(segments,config_dir+'stringer_surface_demo.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  2
DMP for segment  1  of  2
DMP for segment  2  of  2
Time to learn model: 3.2829768657684326 seconds


In [11]:
############################
# Execute stringer model   #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([-0.01552419880890722, 0.012337583487942934, 0.701729727486363, 0.7121672365722354])
t_surf = np.array([0.7122307723105765, -0.19370274318077763, 0.10177554657913149])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf)

not hybrid
hybrid
not hybrid


In [2]:
# Behavior two: Three passes on the layup2 surface (for testing without running sanding tool)

surfaceBSplineFile = '/home/mike/Documents/panda_uli_workspace/src/panda_uli_demo/ULIConfig/registration_models/layup_tool2_surface_demo.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 10 # degrees
R_tool_surf = ScipyR.from_euler('y',ang_relative_to_surface,degrees=True)
q_ts = R_tool_surf.as_quat()



# Part 1: approach the starting point
segment = HybridSegment()
segment.hybrid = False
segment.num_samples = 100 # 2 seconds
segment.state_names = ['x','y','z','qx','qy','qz','qw']

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.2, 0.2)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()

print("QAPP: ",q_app)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.05, 0.0, 0.0, 0.0, 1.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.0, q_app[0], q_app[1], q_app[2], q_app[3]]
orig_vals = interpMultD(starting,ending,segment.num_samples)


# number of state variables by number of samples

segment.original_vals = []
segment.original_vals.append(orig_vals)

segments.append(segment)


# Part 2: one pass over surface (for now)
segment = HybridSegment()
segment.hybrid = True
segment.surface = surfaceBSpline
segment.num_samples = 500 # 4 seconds
segment.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw']
segment.original_vals = []


samps = int(segment.num_samples / 5)
waypoint_0 = interpMultD([0.2, 0.2, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.8, 0.2, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)
waypoint_1 = interpMultD([0.8, 0.2, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.8, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)
waypoint_2 = interpMultD([0.8, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.2, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)
waypoint_3 = interpMultD([0.2, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.2, 0.8, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)
waypoint_4 = interpMultD([0.2, 0.8, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],[0.8, 0.8, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3]],samps)

waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2,waypoint_3,waypoint_4])

segment.original_vals.append(waypoints)

segments.append(segment)


# # Part 3: move away from the surface
segment = HybridSegment()
segment.hybrid = False
segment.num_samples = 100 # 2 seconds
segment.state_names = ['x','y','z','qx','qy','qz','qw']
segment.original_vals = []

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.8, 0.8)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()

starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.005, q_ret[0], q_ret[1], q_ret[2], q_ret[3]]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0]
orig_vals = interpMultD(starting,ending,segment.num_samples)
segment.original_vals.append(orig_vals)

segments.append(segment)



# from cProfile import Profile
# from pstats import Stats

# p = Profile()
# p.enable()
# <stuff>
# p.disable()
# Stats(p).sort_by('cumtime').print_stats()



QAPP:  [ 0.00243397 -0.1279647   0.00551956  0.99176038]


In [3]:
############################
# Learn Layup2 model       #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('corrective_shared_autonomy')+'/../../ULIConfig/registration_models/'


model = DMPLWRhardcoded(verbose=True)
learnedSegments = model.learnModel(segments,config_dir+'layup_tool2_surface_demo.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  2
DMP for segment  1  of  2
DMP for segment  2  of  2
Time to learn model: 10.132977724075317 seconds


In [4]:
############################
# Execute model            #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([-0.012998855441079739, 0.0038797442750289337, 0.6952085803714532, 0.7186800450266385])
t_surf = np.array([0.7386642513098021, -0.14878377910695914, 0.05452037505308755])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf)



[WARN] [1649115455.884446]: Could not process inbound connection: topic types do not match: [geometry_msgs/Twist] vs. [std_msgs/Float64MultiArray]{'callerid': '/ros_uli_wrapper', 'md5sum': '9f195f881246fdfa2798d1d3eebca84a', 'message_definition': '# This expresses velocity in free space broken into its linear and angular parts.\nVector3  linear\nVector3  angular\n\n================================================================================\nMSG: geometry_msgs/Vector3\n# This represents a vector in free space. \n# It is only meant to represent a direction. Therefore, it does not\n# make sense to apply a translation to it (e.g., when applying a \n# generic rigid transformation to a Vector3, tf2 will only apply the\n# rotation). If you want your data to be translatable too, use the\n# geometry_msgs/Point message instead.\n\nfloat64 x\nfloat64 y\nfloat64 z', 'tcp_nodelay': '0', 'topic': '/correction', 'type': 'geometry_msgs/Twist'}
[WARN] [1649115456.887201]: Could not process inbound

not hybrid
hybrid


[WARN] [1649115459.890782]: Could not process inbound connection: topic types do not match: [geometry_msgs/Twist] vs. [std_msgs/Float64MultiArray]{'callerid': '/ros_uli_wrapper', 'md5sum': '9f195f881246fdfa2798d1d3eebca84a', 'message_definition': '# This expresses velocity in free space broken into its linear and angular parts.\nVector3  linear\nVector3  angular\n\n================================================================================\nMSG: geometry_msgs/Vector3\n# This represents a vector in free space. \n# It is only meant to represent a direction. Therefore, it does not\n# make sense to apply a translation to it (e.g., when applying a \n# generic rigid transformation to a Vector3, tf2 will only apply the\n# rotation). If you want your data to be translatable too, use the\n# geometry_msgs/Point message instead.\n\nfloat64 x\nfloat64 y\nfloat64 z', 'tcp_nodelay': '0', 'topic': '/correction', 'type': 'geometry_msgs/Twist'}
[WARN] [1649115463.899350]: Could not process inbound

not hybrid


[WARN] [1649115471.908015]: Could not process inbound connection: topic types do not match: [geometry_msgs/Twist] vs. [std_msgs/Float64MultiArray]{'callerid': '/ros_uli_wrapper', 'md5sum': '9f195f881246fdfa2798d1d3eebca84a', 'message_definition': '# This expresses velocity in free space broken into its linear and angular parts.\nVector3  linear\nVector3  angular\n\n================================================================================\nMSG: geometry_msgs/Vector3\n# This represents a vector in free space. \n# It is only meant to represent a direction. Therefore, it does not\n# make sense to apply a translation to it (e.g., when applying a \n# generic rigid transformation to a Vector3, tf2 will only apply the\n# rotation). If you want your data to be translatable too, use the\n# geometry_msgs/Point message instead.\n\nfloat64 x\nfloat64 y\nfloat64 z', 'tcp_nodelay': '0', 'topic': '/correction', 'type': 'geometry_msgs/Twist'}
[WARN] [1649115487.924209]: Could not process inbound

[WARN] [1649115744.177473]: Could not process inbound connection: topic types do not match: [geometry_msgs/Twist] vs. [std_msgs/Float64MultiArray]{'callerid': '/ros_uli_wrapper', 'md5sum': '9f195f881246fdfa2798d1d3eebca84a', 'message_definition': '# This expresses velocity in free space broken into its linear and angular parts.\nVector3  linear\nVector3  angular\n\n================================================================================\nMSG: geometry_msgs/Vector3\n# This represents a vector in free space. \n# It is only meant to represent a direction. Therefore, it does not\n# make sense to apply a translation to it (e.g., when applying a \n# generic rigid transformation to a Vector3, tf2 will only apply the\n# rotation). If you want your data to be translatable too, use the\n# geometry_msgs/Point message instead.\n\nfloat64 x\nfloat64 y\nfloat64 z', 'tcp_nodelay': '0', 'topic': '/correction', 'type': 'geometry_msgs/Twist'}
[WARN] [1649115776.209370]: Could not process inbound

In [2]:
# Behavior three: Three passes on the layup2 surface (slow with corrections)

import rospkg
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


surfaceBSplineFile = uliconfigpath+'/registration_models/layup_tool2_surface_demo.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 10 # degrees
R_tool_surf = ScipyR.from_euler('y',ang_relative_to_surface,degrees=True)
q_ts = R_tool_surf.as_quat()


# Part 1: approach the starting point
segment1 = HybridSegment()
segment1.hybrid = False
segment1.num_samples = 200 # 2 seconds
segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.2, 0.2)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()

print("QAPP: ",q_app)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.05, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.0, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
orig_vals = interpMultD(starting,ending,segment1.num_samples)


# number of state variables by number of samples

segment1.original_vals = []
segment1.original_vals.append(orig_vals)

segments.append(segment1)


# Part 2: one pass over surface (for now)
segment2 = HybridSegment()
segment2.hybrid = True
segment2.surface = surfaceBSpline
segment2.num_samples = 800 # 8 seconds
segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
segment2.original_vals = []

R_extra_tool = ScipyR.from_euler('y',10,degrees=True)
q_extra = R_extra_tool.as_quat()
corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -0.95, 0.0]
segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples))


samps = int(segment2.num_samples / 5)
waypoint_0 = interpMultD([0.2, 0.2, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.8, 0.2, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps)
waypoint_1 = interpMultD([0.8, 0.2, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.8, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps)
waypoint_2 = interpMultD([0.8, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.2, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps)
waypoint_3 = interpMultD([0.2, 0.5, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.2, 0.8, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps)
waypoint_4 = interpMultD([0.2, 0.8, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.8, 0.8, -5.0, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps)

waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2,waypoint_3,waypoint_4])

segment2.original_vals.append(waypoints)

segments.append(segment2)


# # Part 3: move away from the surface
segment3 = HybridSegment()
segment3.hybrid = False
segment3.num_samples = 200 # 2 seconds
segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment3.original_vals = []
segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.8, 0.8)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()

starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.005, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment3.num_samples)
segment3.original_vals.append(orig_vals)

segments.append(segment3)



# from cProfile import Profile
# from pstats import Stats

# p = Profile()
# p.enable()
# <stuff>
# p.disable()
# Stats(p).sort_by('cumtime').print_stats()


QAPP:  [ 0.00243397 -0.1279647   0.00551956  0.99176038]


In [3]:
############################
# Learn Layup2 model       #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('uli_config')+'/registration_models/'


model = DMPLWRhardcoded(verbose=True)
learnedSegments = model.learnModel(segments,config_dir+'layup_tool2_surface_demo.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  2
DMP for segment  1  of  2
DMP for segment  2  of  2
Time to learn model: 3.447256565093994 seconds
seg time:  3.933906555175781e-05
seg time:  0.6477739810943604
seg time:  2.86102294921875e-05
seg time:  0.00041604042053222656
seg time:  0.6574265956878662
seg time:  0.0004923343658447266


In [9]:
############################
# Execute model            #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([0.00263562868379795, 0.0037402736321456923, 0.7025148096105917, 0.711654414791612])
t_surf = np.array([0.7201873467955325, -0.14119967410462786, 0.04773563206851919])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf, input_type='1dof')

not hybrid
hybrid
not hybrid
